In [ ]:
import pandas as pd
import stumpy
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as dates
from matplotlib.patches import Rectangle
import datetime as dt

In [ ]:
# Load data
df = pd.read_csv(r'C:\Users\USER\Desktop\Master_Irrigation\03_GIS\Flurstuecke\ruten_rigo_dataframe.csv').drop('Unnamed: 0', axis=1)
# Create df's for every Flurstück
dfs = [(x[0], x[1][x[1].date.between('2019-11-01', '2020-07-01')].reset_index(drop=True)) for x in df.groupby('Flurnummer')]

In [ ]:
# Find window size
for flurnr in dfs:
    print(f'Mean time difference between obversations [{flurnr[0]}]: {pd.to_datetime(flurnr[1].date).diff().mean()}')

In [ ]:
m = 20
mp = stumpy.stump(dfs[0][1]['VV_scale'], m)

In [ ]:
motif_idx = np.argsort(mp[:, 0])[0]
print(f'The motif is located at index {motif_idx}')

nearest_neighbor_idx1 = mp[motif_idx, 1]
print(f"The nearest neighbor is located at index {nearest_neighbor_idx}")

nearest_neighbor_idx2 = mp[motif_idx, 2]
print(f"The second nearest neighbor is located at index {nearest_neighbor_idx}")

nearest_neighbor_idx3 = mp[motif_idx, 3]
print(f"The third nearest neighbor is located at index {nearest_neighbor_idx}")

In [ ]:
def find_motifs(dataframes, window_sizes, columns):
    mp_list = list()
    motifs_idx = list()
    nearest_neighbor_idx = list()
    for m in window_sizes:
        for df in dataframes:
            for label in columns:
                mp = stumpy.stump(df[1][label], m, normalize=True)
                mp_list.append((df[0], m, mp, label))
                mo_idx = np.argsort(mp[:, 0])[0]
                motifs_idx.append((df[0], m, mo_idx, df[1].loc[mo_idx, 'date']))
                nn = [mp[motif_idx, 1], mp[motif_idx, 2], mp[motif_idx, 3]]
                nearest_neighbor_idx.append((df[0], m, nn[0], nn[1], nn[2], df[1].loc[nn[0], 'date'], df[1].loc[nn[1], 'date'], df[1].loc[nn[2], 'date']))
    
    df = pd.DataFrame({'label' : [x[3] for x in mp_list],
                  'window_size' : [x[1] for x in mp_list], 
                  'Flurnummer' : [x[0] for x in mp_list],
                  'motif_idx' : [x[2] for x in motifs_idx],
                  'motif_idx_date' : [x[3] for x in motifs_idx],
                  'nearest_neighbor_idx_1' : [x[2] for x in nearest_neighbor_idx],
                  'nearest_neighbor_idx_1_date' : [x[5] for x in nearest_neighbor_idx],
                  'nearest_neighbor_idx_2' : [x[3] for x in nearest_neighbor_idx],
                  'nearest_neighbor_idx_1_date' : [x[6] for x in nearest_neighbor_idx],
                  'nearest_neighbor_idx_3' : [x[4] for x in nearest_neighbor_idx],
                  'nearest_neighbor_idx_1_date' : [x[7] for x in nearest_neighbor_idx],
                  'matrix_profile' : [x[2] for x in mp_list]
                  })
    return df
                              
motifs = find_motifs(dataframes = dfs, 
                     window_sizes = [3,4,5,6,7,8,9,10], 
                     columns = ['VV_scale', 'VH_scale', 'VV_grid', 'VH_grid', 'NDVI_scale', 'NDVI_grid'])

In [ ]:
motifs.drop_duplicates(['Flurnummer', 'motif_idx', 'nearest_neighbor_idx_1', 'nearest_neighbor_idx_2', 'nearest_neighbor_idx_3'])

In [ ]:
def minmax_norm(df_input):
    return (df - df.min()) / ( df.max() - df.min())

def mean_norm(df_input):
    return df_input.apply(lambda x: (x-x.mean())/ x.std(), axis=0)

def quantile_norm(df_input):
    sorted_df = pd.DataFrame(np.sort(df_input.values,axis=0), index=df_input.index, columns=df_input.columns)
    mean_df = sorted_df.mean(axis=1)
    mean_df.index = np.arange(1, len(mean_df) + 1)
    quantile_df =df_input.rank(method="min").stack().astype(int).map(mean_df).unstack()
    return(quantile_df)

In [ ]:
fig, axs = plt.subplots(2, sharex=True, gridspec_kw={'hspace': 0}, dpi=300)
plt.suptitle('Motif (Pattern) Discovery ', fontsize='30')
n_df = mean_norm(dfs[0][1][['VV_scale', 'VH_scale']])
axs[0].plot(n_df['VV_scale'].values)
axs[0].plot(n_df['VH_scale'].values)
min_y = np.min([n_df['VV_scale'].values.min(), n_df['VV_scale'].values.min()])
max_y = np.max([n_df['VH_scale'].values.max(), n_df['VH_scale'].values.max()])
max_y = np.diff([min_y, max_y])
axs[0].set_ylabel('  VV_scale', fontsize='20')
m = motifs.loc[0, 'window_size']
rect = Rectangle((motifs.loc[0, 'motif_idx'], min_y), m, max_y, facecolor='lightgrey', edgecolor='grey', alpha=.7)
axs[0].add_patch(rect)
rect = Rectangle((motifs.loc[0, 'nearest_neighbor_idx_1'], min_y), m, max_y, facecolor='lightblue', edgecolor='blue', alpha=.7)
axs[0].add_patch(rect)
rect = Rectangle((motifs.loc[0, 'nearest_neighbor_idx_2'], min_y), m, max_y, facecolor='lightyellow', edgecolor='yellow', alpha=.7)
axs[0].add_patch(rect)
rect = Rectangle((motifs.loc[0, 'nearest_neighbor_idx_3'], min_y), m, max_y, facecolor='lightgreen', edgecolor='green', alpha=.7)
axs[0].add_patch(rect)

axs[1].set_xlabel('Row Index [time]', fontsize ='20')
axs[1].set_ylabel('Matrix Profile', fontsize='20')
axs[1].axvline(x=motifs.loc[0, 'motif_idx'], linestyle="dashed", color = 'grey' )
axs[1].axvline(x=motifs.loc[0, 'nearest_neighbor_idx_1'], linestyle="dashed", color = 'blue')
axs[1].axvline(x=motifs.loc[0, 'nearest_neighbor_idx_2'], linestyle="dashed", color = 'yellow')
axs[1].axvline(x=motifs.loc[0, 'nearest_neighbor_idx_3'], linestyle="dashed", color = 'green')

axs[1].plot(motifs.loc[0, 'matrix_profile'][:, 0])

plt.show()